In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
from statistics import mean
import numpy as np

# 1. ***Data***

## **Data Preprocessing**

As always, in the data science area, you can find some inconsistencies in the provided data. Therefore, some modifications should be made to the data to make it consistent across all of the datasets you have. To ensure consistency in the data, keep the following in mind:

  1)  Some of the heroes' names in 'hero-network.csv' are not found in 'edges.csv'. This inconsistency exists for the following reasons:
        - Some heroes' names in 'hero-netowrk.csv' have extra spaces at the end of their names compared to their names in 'edges.csv'.
        - Some heroes' names in 'hero-netowrk.csv' have an extra '/' at the end of their names compared to their names in 'edges.csv'.
        - The hero name 'SPIDER-MAN/PETER PARKER' in 'edges.csv' has been changed to 'SPIDER-MAN/PETER PAR' in 'hero-network.csv' due to a string length    limit in 'hero-network.csv'.


  2)  Some entries in the 'hero-network.csv' have the same hero in both columns. In the graph, these entries form a self-loop. Because a self-loop makes no sense in this network, you can safely remove those from the dataset.


In [2]:
hero_df = pd.read_csv('hero-network.csv')
edges_df = pd.read_csv('edges.csv')
nodes_df = pd.read_csv('nodes.csv')

We decided to homogenize all the DFs since there are many words that are the same but with extra characters in the end
and in the hero-network's DF there is a character limit of 20 digits.

We report here some examples of the issues we could find

In [3]:
print('SPIDER-MAN/PETER PAR' in hero_df.values)         #digits limit
print('SPIDER-MAN/PETER PARKERKER' in nodes_df.values)  #with an extra "KER" in the end
print('SPIDER-MAN/PETER PARKER' in edges_df.values)     

True
True
True


Trimming to the 20th character and removing final blank spaces or final "/" character

In [4]:

hero_df['hero1'] = hero_df['hero1'].apply(lambda x: x.rstrip('/ ')[:20])
hero_df['hero2'] = hero_df['hero2'].apply(lambda x: x.rstrip('/ ')[:20])
edges_df['hero'] = edges_df['hero'].apply(lambda x: x.rstrip('/ ')[:20])
edges_df['comic'] = edges_df['comic'].apply(lambda x: x.rstrip('/ ')[:20])
nodes_df['node'] = nodes_df['node'].apply(lambda x: x.rstrip('/ ')[:20])


Let's check if the are any anomalies in our data now that we have formatted them.

In [5]:
print('SPIDER-MAN/PETER PAR' in hero_df.values)         #digits limit
print('SPIDER-MAN/PETER PARKERKER' in nodes_df.values)  #with an extra "KER" in the end
print('SPIDER-MAN/PETER PARKER' in edges_df.values)

True
False
False


Now let's remove the rows from the hero_df wich has the same hero in both columns.

In [6]:
hero_df.drop(hero_df[hero_df['hero1'] == hero_df['hero2']].index, inplace = True)

## **Graphs setup**

First graph: Will be constructed using the data stored in the 'hero-network.csv' file, in which an edge between two heroes can be found if they have appeared in the same comic together. The number of edges between two heroes represents the number of times they have collaborated in different comics. The graph should be considered weighted and undirected. It is up to you to decide which metric to use to calculate the weights, but we anticipate that the cost will be lower for heroes with more collaborations. Please specify which metric you used to select the weights in the report.

We group by the hero1 and hero2 on the hero_df to obtain the number of occurencies between to nodes, which corresponds to the number of times a hero collaborated with another hero. The we obtain a new dataset from this operation, in fact we need the 'weight' variable in our dataset.

In [7]:
count_hero_df = hero_df.groupby(["hero1", "hero2"]).size().reset_index(name='weight')

We set the weight to be 1/n because it has to be inversely related to the number of times each hero collaborted with another hero. In this way heroes who got lot of collaborations will be penalized.

In [8]:
count_hero_df['weight'] = count_hero_df['weight'].apply(lambda x: 1/x)

In [9]:
count_hero_df

,hero1,hero2,weight
0,24-HOUR MAN/EMMANUEL,"FROST, CARMILLA",1.0
1,24-HOUR MAN/EMMANUEL,KILLRAVEN/JONATHAN R,1.0
2,24-HOUR MAN/EMMANUEL,M'SHULLA,1.0
3,3-D MAN/CHARLES CHAN,ANGEL/WARREN KENNETH,1.0
4,3-D MAN/CHARLES CHAN,ANT-MAN II/SCOTT HAR,1.0
...,...,...,...
224094,ZZZAX,"RODRIGUEZ, DEBRA",1.0
224095,ZZZAX,"ROSS, GEN. THADDEUS",0.5
224096,ZZZAX,"SUMMERS, NATHAN CHRI",1.0
224097,ZZZAX,TIGRA/GREER NELSON,1.0


Now we can create a 'weighted graph' using the weight we calculated as our edge attribute.

In [10]:
G_hero_net = nx.from_pandas_edgelist(count_hero_df, 'hero1', 'hero2', edge_attr='weight', create_using=nx.MultiGraph)

In [11]:
G_hero_net['ZZZAX']['RODRIGUEZ, DEBRA']

AtlasView({0: {'weight': 1.0}})

**Second graph** : The data in 'nodes.csv' and 'edges.csv' will be used to construct the second graph. The type of node (hero/comic) can be found in 'nodes.csv', and an edge between a hero node and a comic node can be found in 'edges.csv' when the hero has appeared in that specific comic. This graph is assumed to be undirected and unweighted.

We need to set the 'node' column as index of the nodes_df dataframe in order to proceed with our request.


In [12]:
print(len(nodes_df))
print(len(nodes_df.node.unique()))

19090
19087


We see that there are only 3 non unique values in the 'node' column se we decided to drop them to mantain the dataframe consistency.

In [13]:
nodes_df = nodes_df.drop_duplicates(subset=['node'])

We transform the node column in the new index of the dataframe and we transform it in a dictionary.

In [14]:
nodes_attr = nodes_df.set_index('node').to_dict(orient = 'index')

Now we can proceed with the creation of the second graph. First we create a new graph starting from the edges dataframe and then we select the nodes_attr dictionary to be the attributes of our nodes.

In [15]:
G_edges_net = nx.from_pandas_edgelist(edges_df, 'hero', 'comic')
nx.set_node_attributes(G_edges_net, nodes_attr)

In [16]:
G_edges_net.nodes['VENUS II']

{'type': 'hero'}

# 2. ***Backend Implementation***

The goal of this part is the implementation of a controller system that has different functionalities. The controller should take as input an identifier "i" and run the associated function_i applied to the graph you create from the downloaded data.

Definition: As the number of nodes and edges grows, we may request to work on a subset of the data to reduce computation time and improve network visualization. In this case, we will ask you only to consider the data for top N heros. We define the top N heroes as follows:

* Top N heroes: The top N heroes who have appeared in the most number of comics. The 'edges.csv' file, which represents the comics in which each hero has appeared, can be used to filter these N heroes.

Note: When the value of N is not set by the user, the function should consider the whole data.
Functionality 1 - extract the graph's features

## ***Functionality 1 - Extract the graph's features***

**Input**: 
* The graph data
* The graph type (ex number 1 or number 2)
* N: denoting the top N heroes that their data should be considered

**Output**:

* The number of nodes in the network (if type 2, report for both node types)
* The number of collaborations of each superhero with the others (only if type 1)
* The number of heroes that have appeared in each comic (only if type 2)
* The network's density
* The network's degree distribution
* The average degree of the network
* The network's Hubs (hubs are nodes having degrees more extensive than the 95th percentile of the degree distribution)
* Whether the Network is sparse or dense

In [17]:
def TOP_N(N,G_hero_net,edges_df):
    subnodes=set([el for el in edges_df.groupby('hero').count().sort_values(by=['comic'],ascending=False).head(N).index])
    return G_hero_net.subgraph(subnodes)

In [18]:
def Number_of_nodes(graph_type,graph):
    if graph_type == 1 :
        print(f"The graph have " + str(graph.number_of_nodes()) +  " nodes")
        return
    elif graph_type == 2 :
        comic=0
        hero=0
        attr=nx.get_node_attributes(graph,'type')
        for k in graph.nodes:
            if (attr[k]=='comic'):
                comic+=1
            else:
                hero+=1
        print(f"The graph have " + str(comic) +  " nodes of type: COMIC")
        print(f"The graph have " + str(hero) +  " nodes of type: HERO")
        return 
    else:
        return ('Wrong type number')

In [19]:
def number_of_collaborations(graph_type,graph):
    if graph_type!=1:
        print('Valid only for type 1')
        return
    else:
        for node in graph:
            print( node + " has " + str(graph.degree(node)) + " collaborations.")
        return
    

In [20]:
def number_of_hero_in_each_comic(graph_type):
    if graph_type!=2:
        print('number_of_hero_in_each_comic: Valid only for type 2')

        return
    else:
        attr=nx.get_node_attributes(G_edges_net,'type')
        List_hero=[i for i in G_edges_net.nodes if attr[i]=='hero']
        tmp=0
        for hero in List_hero:
            if G_edges_net.degree(hero)==12651:
                print( hero + " has appeared in each comic")
                tmp+=1
        return tmp

In [21]:
def degree_distribution(graph):
    deg = [graph.degree(n) for n in graph.nodes()]
    deg_count = Counter(deg)
    return deg_count

In [22]:
def degree_mean(graph):
    deg = [graph.degree(n) for n in graph.nodes()]
    return mean(deg)

In [23]:
def nodes_hubs(graph):
    deg = [graph.degree(n) for n in graph.nodes()]
    percentile = np.percentile(deg, 95)

    return [n for n in graph.nodes() if graph.degree(n) > percentile]

In [24]:
def graph_density(graph):
    nodes_count = graph.number_of_nodes()
    max_edges = nodes_count * (nodes_count - 1) / 2
    edges_count = graph.number_of_edges()
    density = edges_count/ max_edges
    return density

In [25]:
def Functionality_1(graph,graph_type,N=6421): 
    if (N!=6421 and graph_type==1):
        graph=TOP_N(N,G_hero_net,edges_df)
    
    Number_of_nodes(graph_type,graph)
    print('\n')
    number_of_collaborations(graph_type,graph)
    print('\n')
    number_of_hero_in_each_comic(graph_type)
    print('\n')
    dens=graph_density(graph)
    print('The degree distribution is',degree_distribution(graph))
    print('\n')
    print('The average degree is',degree_mean(graph))
    print('\n')
    print('The node hubs are',nodes_hubs(graph))
    print('\n')
    print('The density is',dens)
    print('\n')
    if dens>=0.5:
        print('The graph is dense')
    else:
        print('The graph is sparse')
    
    return

In [26]:
Functionality_1(G_hero_net,1,N=10)

The graph have 10 nodes


HULK/DR. ROBERT BRUC has 18 collaborations.
THING/BENJAMIN J. GR has 18 collaborations.
THOR/DR. DONALD BLAK has 18 collaborations.
MR. FANTASTIC/REED R has 18 collaborations.
INVISIBLE WOMAN/SUE has 18 collaborations.
SPIDER-MAN/PETER PAR has 18 collaborations.
CAPTAIN AMERICA has 18 collaborations.
WOLVERINE/LOGAN has 18 collaborations.
HUMAN TORCH/JOHNNY S has 18 collaborations.
IRON MAN/TONY STARK has 18 collaborations.


number_of_hero_in_each_comic: Valid only for type 2


The degree distribution is Counter({18: 10})


The average degree is 18


The node hubs are []


The density is 2.0


The graph is dense


## Functionality 2 - Find top superheroes!


*Input*:
* The graph data
* A node (hero or comic)
* One of the given metrics : *Betweeness, PageRank, ClosenessCentrality, DegreeCentrality*
* N: denoting the top N heroes that their data should be considered

*Output*:
* The metric's value over the considered graph
* The given node's value

Note: Give an explanation regarding the features of the user based on all of the metrics (e.g. if the betweenness metric is high, what does this mean in practice, what if the betweenness is low but has a high PageRank value, etc.).

###  Metrics

* **Degree centrality**:

   The degree centrality of a node in a graph is a measure of the importance of that node in the graph. It is defined as the number of edges incident to the node, i.e., the number of neighbors it has. In other words, it is the number of connections a node has in the graph. The degree centrality of a node can be calculated as:
   
   $ Degree\,Centrality = \frac{number\,\,of\,\,neighbors}{number\,of\,nodes\, -\, 1} $

   Nodes with high degree centrality are often considered important or influential in the graph, as they have a large number of connections and may be able to spread information or influence other nodes more effectively.


* **Betweenness centrality**:

   The betweenness centrality of a node in a graph is a measure of the importance of that node in terms of the number of shortest paths that pass through it. It is defined as the fraction of all shortest paths in the graph that pass through the node. In other words, it is a measure of the node's centrality in terms of the connectivity of the graph.
   The betweenness centrality of a node can be calculated as:

   $ Betweenness\,centrality = \frac{num\,of\,shortest\,path\,passing\,through\,the\,node}{total\,num\,of\,shortest\,paths\,in\,the\,graph}$
   
   Nodes with high betweenness centrality are often considered important or influential in the graph, as they are more likely to be on the shortest path between other nodes and may be able to control the flow of information or influence other nodes more effectively.

* **Closeness centrality** :

    The closeness centrality of a node in the graph is a measure of the importance of that node in terms of its ability to reach other nodes in the graph. It is defined as the reciprocal of the sum of the shortest path distances from the node to all other nodes in the graph. In other words, it is a measure of the node's centrality in terms of its proximity to other nodes.The closeness centrality can be calculated as:

    $ Closeness\,centrality = \frac{1}{sum\,of\,the\,shortest\,path\,distances\,from\,the\,node\,to\,all\,other\,nodes}$ 

    Nodes with high closeness centrality are often considered important or influential in the graph, as they are able to reach other nodes quickly and may be able to spread information or influence other nodes more effectively.

* **Page rank**:

    The page rank algorithm is a way to measuring the importance or ranking of nodes in a network. In the context of a network, the PageRank of a node is a measure of the importance of that node based on the number and quality of the edges pointing to it. Nodes with a higher PageRank are considered more important and are more likely to appear at the top of search results.
    
    To calculate the PageRank of the nodes in a network, the algorithm follows these steps:
    * First of all we need to transform our graph in a directed graph.
    * We need to assign a initial PageRank score to each node.
    * Then we need to iteratively update the PageRank score of each node based on the PageRank score of the nodes that link to it. Specifically, the PageRank of a node is equal to the sum of the PageRanks of the nodes that link to it, divided by the number of outbound links from each of those nodes.
    We also need to use a damping factor in order to avoid infinite loops and allow the algorithm to converge.
    * We need to repeat the previous step until the score converge.

### Correlation between different metrics

The correlation between different network metrics will depend on the specific characteristics of the network and the definition of the metrics. In general, different metrics may capture different aspects of the network and may not be strongly correlated.

For example, **degree centrality** measures the number of connections a node has, while **betweenness centrality** measures the number of times a node acts as a "bridge" between other nodes in the network. These two metrics may not be strongly correlated, as they capture different aspects of the network.

In the same way, also the **closeness centrality** capture another different aspect of the network, in fact this metric is a measure of the average distance of a node to all other nodes in the network. High closeness centrality indicates a node that is considered to be central to our network.

Similarly, **PageRank**, which is a measure of the importance or ranking of a node based on the number and quality of the links pointing to it, may not be strongly correlated with other metrics such as degree centrality or betweenness centrality.

In [53]:
#TODO: Fasten up betweeness, closeness and degree if it is possible.

In [39]:
damping_factor = 0.85

def compute_pagerank(graph, pagerank):
    for node in graph:
        rank = 1- damping_factor
        for neighbor in graph[node]:
            rank += damping_factor * pagerank[neighbor] / len(graph[neighbor])
            pagerank[node] = rank
    return pagerank

In [40]:
def page_rank_metric(graph):
    directed_graph = graph.to_directed()
    pagerank = {node: 1/len(directed_graph) for node in directed_graph}
    while True:
        prev_rank = pagerank.copy()
        pagerank = compute_pagerank(directed_graph, pagerank)
        if all(abs(prev_rank[node] - pagerank[node]) < 0.001 for node in pagerank):
            break
    return (pagerank)

In [27]:
def betweeness_metric(graph): #MAKE SURE TO CONSIDER ONLY A SUBGRAPH BECAUSE THIS ALGORITHM IN VERY SLOW
    betweeness = {n: 0 for n in graph.nodes()}
    for node in graph.nodes():
        for destin in graph.nodes():
            if destin!=node:
                paths = list(nx.all_shortest_paths(graph, node, destin))
                for key in paths:
                    for value in paths:
                        if key!=value:
                            for n in paths[(key, value)]:
                                betweeness[n] += 1 / len(paths[(key, value)])
    return betweeness


In [28]:
def closeness_metric(graph):
    closeness = {n: 0 for n in graph.nodes()}
    for node in graph.nodes():
        count=0
        for destin in graph.nodes():
            if destin!=node:
                count += len(list(nx.all_shortest_paths(graph, node, destin)))
        closeness[node]=(len(graph.nodes())-1)/count   
    return closeness

In [29]:
def degree_metric(graph):
    deg={n: 0 for n in graph.nodes()}
    for node in graph.nodes():
        deg[node]=graph.degree[node]/(len(graph.nodes())-1)
    return deg #IF YOU COMPARE IT WITH THE BUILT IN FUNCTION THERE IS A DIFFERENCE OF e-16 IN THE VALUES, BASICALLY ARE THE SAME

In [48]:
def functionality_2(graph, node, metric, N=6421): #DO NOT USE IT WITH A BIG N, IT IS TOO SLOW
    
    if N!=6421:
        graph=TOP_N(N,G_hero_net,edges_df)
    print('The '+str(metric)+' is','\n')
    
    if metric.lower()=='betweeness':
        print(betweeness_metric(graph))
        print('\n')
        print('The value for the node '+str(node)+' is: ',betweeness_metric(graph)[node])
    elif metric.lower()=='pagerank':
        print(page_rank_metric(graph))
        print('\n')       
        print('The value for the node '+str(node)+' is: ',page_rank_metric(graph)[node])
    elif metric.lower()=='closenesscentrality':
        print(closeness_metric(graph))
        print('\n')
        print('The value for the node '+str(node)+' is: ',closeness_metric(graph)[node])        
    elif metric.lower()=='degreecentrality':
        print(degree_metric(graph)) 
        print('\n')
        print('The value for the node '+str(node)+' is: ',degree_metric(graph)[node])         
    else:
        print('This metric is not included')
    return
   
    

In [51]:
functionality_2(G_hero_net, 'SCARLET WITCH/WANDA', 'pagerank', 45)

The pagerank is 

{'PROFESSOR X/CHARLES': 0.992902705755964, 'WONDER MAN/SIMON WIL': 1.0128870934264516, 'JARVIS, EDWIN': 1.0129021535930094, 'WATSON-PARKER, MARY': 0.9533709591131375, 'INVISIBLE WOMAN/SUE': 1.0129318272448404, 'SPIDER-MAN/PETER PAR': 1.012946573282, 'JAMESON, J. JONAH': 0.9930006977227123, 'SHADOWCAT/KATHERINE': 0.9137379049089402, 'CYCLOPS/SCOTT SUMMER': 1.0129901701248862, 'QUICKSILVER/PIETRO M': 0.9931763875941791, 'CANNONBALL II/SAM GU': 0.9338999796954497, 'ODIN [ASGARDIAN]': 0.9141324939098797, 'FURY, COL. NICHOLAS': 0.9931561950713027, 'ANGEL/WARREN KENNETH': 1.0130609036405598, 'PARKER, MAY': 0.9337897479962868, 'ICEMAN/ROBERT BOBBY': 1.013088569464303, 'SHE-HULK/JENNIFER WA': 1.0131022148277247, 'HULK/DR. ROBERT BRUC': 1.0131157643708038, 'THING/BENJAMIN J. GR': 1.013129218766409, 'ANT-MAN/DR. HENRY J.': 1.013142578682684, 'HERCULES [GREEK GOD]': 0.9932511403083025, 'SCARLET WITCH/WANDA': 1.0131690498924886, 'HUMAN TORCH/JOHNNY S': 1.013182130107005, "BLACK P

In [52]:
functionality_2(G_hero_net,'SCARLET WITCH/WANDA','DegreeCentrality',45)

The DegreeCentrality is 

{'PROFESSOR X/CHARLES': 1.9090909090909092, 'WONDER MAN/SIMON WIL': 1.9090909090909092, 'JARVIS, EDWIN': 1.9318181818181819, 'WATSON-PARKER, MARY': 1.6363636363636365, 'INVISIBLE WOMAN/SUE': 2.0, 'SPIDER-MAN/PETER PAR': 2.0, 'JAMESON, J. JONAH': 1.9090909090909092, 'SHADOWCAT/KATHERINE': 1.7272727272727273, 'CYCLOPS/SCOTT SUMMER': 1.9545454545454546, 'QUICKSILVER/PIETRO M': 1.9090909090909092, 'CANNONBALL II/SAM GU': 1.5909090909090908, 'ODIN [ASGARDIAN]': 1.5454545454545454, 'FURY, COL. NICHOLAS': 1.9318181818181819, 'ANGEL/WARREN KENNETH': 1.9318181818181819, 'PARKER, MAY': 1.4318181818181819, 'ICEMAN/ROBERT BOBBY': 2.0, 'SHE-HULK/JENNIFER WA': 1.9772727272727273, 'HULK/DR. ROBERT BRUC': 2.0, 'THING/BENJAMIN J. GR': 2.0, 'ANT-MAN/DR. HENRY J.': 1.9772727272727273, 'HERCULES [GREEK GOD]': 1.9545454545454546, 'SCARLET WITCH/WANDA': 1.9772727272727273, 'HUMAN TORCH/JOHNNY S': 2.0, "BLACK PANTHER/T'CHAL": 1.9318181818181819, 'DR. STRANGE/STEPHEN': 1.977272727272

## Functionality 3 - Shortest ordered Route

**Input**:

* The graph data
* A sequence of superheroes h = [h_2, ..., h_n-1]
* Initial node h_1 and an end node h_n
* N: denoting the top N heroes that their data should be considered

**Output**:

* The shortest walk of comics that you need to read to get from hero_1 to hero_n

Considerations: For this functionality, you need to implement an algorithm that returns the shortest walk that goes from node h_j to h_n, which visits in order the nodes in h. The choice of h_j and h_n can be made randomly (or if it improves the performance of the algorithm, you can also define it in any other way)

**Important Notes**:

* This algorithm should be run only on the second graph.
* The algorithm needs to handle the case that the graph is not connected. Thus, only some of the nodes in h are reachable from h_1. In such a scenario, it is enough to let the program give in the output the string "There is no such path".
* Since we are dealing with walks, you can pass on the same node h_i more than once, but you have to preserve order. E.g., if you start from Spiderman to reach deadpool, and your path requires you to visit iron-man and colossus, you can go back to any comics any time you want, assuming that the order in which you visit the heroes is still the same.
